In [6]:
import pandas as pd
from IPython.display import display
import numpy as np
from matplotlib import rc
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from tqdm import tqdm
import pyCompare
import re

from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.set_option("display.max.columns", None)

plt.rcParams['figure.figsize'] = (10, 10)

In [7]:
df = pd.read_csv('./data_preop.csv')
df.head()

,caseid,age,sex,bmi,asa,emop,department,optype,dx,opname,approach,ane_type,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,cormack,airway,aline1,aline2,cline1,cline2,intraop_uo,intraop_rbc,intraop_ffp,intraop_crystalloid,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca,WBC,Hb,PLT,Albumin,GOT (AST),GPT (ALT),Creatinine,Sodium,Potassium,Chloride,hs-CRP quantitation,PT (INR),aPTT,Fibrinogen,pH,pCO₂,pO₂,HCO3-,BE,Calcium,Glucose,Lactic acid
0,1,77.0,M,26.3,2.0,N,General surgery,Colorectal,Rectal cancer,Low anterior resection,Open,General,180,8,0,N,Y,N,N,Normal,14.1,189,94.0,33.2,141.0,3.1,134.0,4.3,18,16,10.0,0.82,I,Oral,Left radial,N,N,N,300.0,0,0,350.0,0,120,0.0,100,70,0,10,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.0,3.1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,54.0,M,19.6,2.0,N,General surgery,Stomach,Advanced gastric cancer,Subtotal gastrectomy,Open,General,245,19,0,N,N,N,N,Normal,10.2,251,110.0,31.9,143.0,4.7,88.0,3.8,18,15,14.0,0.86,I,Oral,N,N,N,N,700.0,0,0,800.0,0,150,0.0,0,100,0,20,0,0,0,4.36,10.2,251.0,NaN,NaN,NaN,NaN,143.0,4.7,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,62.0,M,24.4,1.0,N,General surgery,Biliary/Pancreas,Gallbladder stone,Cholecystectomy,Videoscopic,General,65,2,0,N,N,N,N,Normal,14.2,373,103.0,30.3,144.0,4.9,87.0,4.2,17,34,14.0,1.18,I,Oral,N,N,N,N,NaN,0,0,200.0,0,0,0.0,0,50,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,74.0,M,20.5,2.0,N,General surgery,Stomach,Advanced gastric cancer,Distal gastrectomy,Videoscopic,General,335,7,1,N,Y,N,N,Normal,14.4,275,103.0,34.5,141.0,4.2,108.0,4.1,23,18,10.0,0.96,I,Oral,Right radial,N,N,N,270.0,0,0,2700.0,0,80,0.0,100,100,0,50,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,4.3,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,66.0,M,20.4,3.0,Y,General surgery,Vascular,Aortic aneurysm,Aneurysmal repair,Open,General,350,45,13,N,Y,N,Left anterior fascicular block,Normal,10.1,67,73.0,36.5,146.0,4.4,126.0,2.6,765,77,50.0,4.43,I,Oral,Right radial,N,Right IJV,N,1490.0,8,8,7100.0,0,0,0.0,0,160,0,10,900,0,2100,14.64,15.3,158.0,4.6,64.0,30.0,1.83,139.0,4.9,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.9,140.0,NaN


In [8]:
pd.DataFrame(df.isnull().sum() / len(df), columns=['Null Percentage']).T

,caseid,age,sex,bmi,asa,emop,department,optype,dx,opname,approach,ane_type,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,cormack,airway,aline1,aline2,cline1,cline2,intraop_uo,intraop_rbc,intraop_ffp,intraop_crystalloid,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca,WBC,Hb,PLT,Albumin,GOT (AST),GPT (ALT),Creatinine,Sodium,Potassium,Chloride,hs-CRP quantitation,PT (INR),aPTT,Fibrinogen,pH,pCO₂,pO₂,HCO3-,BE,Calcium,Glucose,Lactic acid
Null Percentage,0.0,0.0,0.0,0.0,0.019709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03827,0.037313,0.039227,0.042289,0.081324,0.081324,0.040949,0.039418,0.038462,0.037696,0.038462,0.040184,0.143131,0.073096,0.0,0.0,0.0,0.0,0.450057,0.0,0.0,0.063911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400115,0.400115,0.403368,0.412744,0.412361,0.412361,0.423268,0.289323,0.289705,0.290088,0.61328,0.474742,0.476081,0.515882,0.950823,0.950823,0.950823,0.950823,0.987754,0.377535,0.484118,0.950823


In [9]:
for i in df.columns:
    if df[i].dtype == object:
        print('Variable: {0}'.format(i))
        print('class: {0} \n'.format(set(df[i])))

Variable: sex
class: {'M', 'F'} 

Variable: emop
class: {'N', 'Y'} 

Variable: department
class: {'Gynecology', 'Urology', 'General surgery'} 

Variable: optype
class: {'Breast', 'Major resection', 'Stomach', 'Colorectal', 'Thyroid', 'Minor resection', 'Vascular', 'Hepatic', 'Others', 'Transplantation', 'Biliary/Pancreas'} 

Variable: dx
class: {'Rectal carcinoid, benign', 'end stage renal disease ', 'Other appendicitis', 'Malignant neoplasm of ileocecal valve', 'Gastrointestinal stromal tumor of rectum, malignant ', 'Intrahepatic duct stone with obstruction', 'Small intestine obstruction', 'Esophageal cancer', 'Left inguinal hernia', 'Neoplasm of appendix, uncertain behavior', 'Atherosclerosis obliteranswithout gangrene', 'Lobular carcinoma in situ of breast unspecified side', 'Testicular disease ', 'Unclassified uterine cancer', 'Leiomyosarcoma ', 'Ileus ', 'Gastric submucosal tumor, benign', "Crohn's disease moderate", 'Cecal cancer', "Wilson' disease ", 'Epithelial ovarian cancer u

## Preprocessing and Cleaning

In [10]:
df['sex'] = df['sex'].map({'F': 0, 'M': 1})
df['emop'] = df['emop'].map({'N': 0, 'Y': 1})
df['death_inhosp'] = df['death_inhosp'].map({'N': 0, 'Y': 1})
df['preop_htn'] = df['preop_htn'].map({'N': 0, 'Y': 1})
df['preop_dm'] = df['preop_dm'].map({'N': 0, 'Y': 1})

# aline & cline
df.loc[~df['aline1'].isin(['N']), 'aline1'] = 1
df.loc[df['aline1'].isin(['N']), 'aline1'] = 0
df.loc[~df['aline2'].isin(['N']), 'aline2'] = 1
df.loc[df['aline2'].isin(['N']), 'aline2'] = 0
df.loc[~df['cline1'].isin(['N']), 'cline1'] = 1
df.loc[df['cline1'].isin(['N']), 'cline1'] = 0
df.loc[~df['cline2'].isin(['N']), 'cline2'] = 1
df.loc[df['cline2'].isin(['N']), 'cline2'] = 0
df['aline'] = df[['aline1', 'aline2']].any(axis=1).astype(int) # 0(no), 1(yes) 
df['cline'] = df[['cline1', 'cline2']].any(axis=1).astype(int) # 0(no), 1(yes)
df.drop(columns=['aline1', 'aline2', 'cline1', 'cline2'],inplace=True)

## preop_arrhythmia

## Numeric Value
def clean_num(df_series):
    df_series = str(df_series)
    num_pattern = re.compile(r'\s*(?P<value>[0-9]+[.]*[0-9]*)\s*')
    re_num = re.search(num_pattern, df_series)
    if re_num != None:
        value = re_num.group('value')
    else:
        value = np.nan
        
    return value

df['preop_plt'] = df['preop_plt'].apply(lambda x: clean_num(x)).astype(float)
df['preop_aptt'] = df['preop_aptt'].apply(lambda x: clean_num(x)).astype(float)
df['preop_got'] = df['preop_got'].apply(lambda x: clean_num(x)).astype(float)
df['preop_gpt'] = df['preop_gpt'].apply(lambda x: clean_num(x)).astype(float)
df['preop_cr'] = df['preop_cr'].apply(lambda x: clean_num(x)).astype(float)


In [11]:
df['aline'].value_counts()

0    2875
1    2351
Name: aline, dtype: int64